In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bank-customer-segmentation/bank_transactions.csv


In [2]:
df=pd.read_csv("/kaggle/input/bank-customer-segmentation/bank_transactions.csv")
df.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


Dropping useless columns

In [3]:
df=df.drop("CustLocation",axis=1)
df=df.drop('TransactionID',axis=1)

In [4]:
df

,CustomerID,CustomerDOB,CustGender,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,C5841053,10/1/94,F,17819.05,2/8/16,143207,25.0
1,C2142763,4/4/57,M,2270.69,2/8/16,141858,27999.0
2,C4417068,26/11/96,F,17874.44,2/8/16,142712,459.0
3,C5342380,14/9/73,F,866503.21,2/8/16,142714,2060.0
4,C9031234,24/3/88,F,6714.43,2/8/16,181156,1762.5
...,...,...,...,...,...,...,...
1048562,C8020229,8/4/90,M,7635.19,18/9/16,184824,799.0
1048563,C6459278,20/2/92,M,27311.42,18/9/16,183734,460.0
1048564,C6412354,18/5/89,M,221757.06,18/9/16,183313,770.0
1048565,C6420483,30/8/78,M,10117.87,18/9/16,184706,1000.0


Cleaning overall - dropping duplicates and nulls

In [5]:
df.dropna(inplace=True)

In [6]:
df

,CustomerID,CustomerDOB,CustGender,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,C5841053,10/1/94,F,17819.05,2/8/16,143207,25.0
1,C2142763,4/4/57,M,2270.69,2/8/16,141858,27999.0
2,C4417068,26/11/96,F,17874.44,2/8/16,142712,459.0
3,C5342380,14/9/73,F,866503.21,2/8/16,142714,2060.0
4,C9031234,24/3/88,F,6714.43,2/8/16,181156,1762.5
...,...,...,...,...,...,...,...
1048562,C8020229,8/4/90,M,7635.19,18/9/16,184824,799.0
1048563,C6459278,20/2/92,M,27311.42,18/9/16,183734,460.0
1048564,C6412354,18/5/89,M,221757.06,18/9/16,183313,770.0
1048565,C6420483,30/8/78,M,10117.87,18/9/16,184706,1000.0


Cleaning age

In [7]:
age=[]

In [8]:
dob=df.iloc[:,1].values

In [9]:
def find_bad(date):
    try:
        year=date.split("/")[-1]
        if(int(year)>=100):
            return date
        
    except:
        pass
    

In [10]:
#Finding bad or corrputed dates
s=set()
for i in dob:
    x=find_bad(i)
    if(x):
        s.add(x)

In [11]:
s

{'1/1/1800'}

In [12]:
df = df.drop(df[df['CustomerDOB'] == '1/1/1800'].index)

In [13]:
def find_age(date):
    year=date.split("/")[-1]
    return int(year)

In [14]:
dob=df.iloc[:,1].iloc
minn=100
for i in dob:
    minn=min(minn,find_age(i))

In [15]:
minn

0

Assuming the data is from 1900->1999 as this data was produced on 2016 and minors are not allowed for transactions

In [16]:
def convert(date):
    year=date.split("/")[-1]
    year="19"+year
    return (2016-int(year))

In [17]:
for i in dob:
    age.append(convert(i))

In [18]:
df["age"]=age

In [19]:
df

,CustomerID,CustomerDOB,CustGender,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),age
0,C5841053,10/1/94,F,17819.05,2/8/16,143207,25.0,22
1,C2142763,4/4/57,M,2270.69,2/8/16,141858,27999.0,59
2,C4417068,26/11/96,F,17874.44,2/8/16,142712,459.0,20
3,C5342380,14/9/73,F,866503.21,2/8/16,142714,2060.0,43
4,C9031234,24/3/88,F,6714.43,2/8/16,181156,1762.5,28
...,...,...,...,...,...,...,...,...
1048562,C8020229,8/4/90,M,7635.19,18/9/16,184824,799.0,26
1048563,C6459278,20/2/92,M,27311.42,18/9/16,183734,460.0,24
1048564,C6412354,18/5/89,M,221757.06,18/9/16,183313,770.0,27
1048565,C6420483,30/8/78,M,10117.87,18/9/16,184706,1000.0,38


# Customer details dataset

In [20]:
customers=pd.DataFrame()
customers["ID"]=df.iloc[:,0].values
customers["Age"]=df.iloc[:,7].values
customers["Gender"]=df.iloc[:,2].values

In [21]:
customers

,ID,Age,Gender
0,C5841053,22,F
1,C2142763,59,M
2,C4417068,20,F
3,C5342380,43,F
4,C9031234,28,F
...,...,...,...
985468,C8020229,26,M
985469,C6459278,24,M
985470,C6412354,27,M
985471,C6420483,38,M


There are conflicting entries, with one customer having two age. so we will find and remove such customers.

In [22]:
details={}
faulty=set()
ids=customers.iloc[:,0].values
age=customers.iloc[:,1].values
gender=customers.iloc[:,2].values

In [23]:
n=ids.shape[0]


In [24]:
for i in range(n):
    cid=ids[i]
    if(cid in details):
        if(details[cid]==(age[i],gender[i])):
            continue
        else:
            del(details[cid])
            faulty.add(cid)
    else:
        if cid in faulty:
            pass
        else:
            details[cid]=(age[i],gender[i])

In [25]:
len(faulty)+len(details)==customers["ID"].nunique()

True

Our checking is right, thus we can remove.

In [26]:
customers=customers[~customers['ID'].isin(faulty)]

In [27]:
customers

,ID,Age,Gender
0,C5841053,22,F
1,C2142763,59,M
2,C4417068,20,F
4,C9031234,28,F
5,C1536588,44,F
...,...,...,...
985466,C5028150,36,M
985467,C1034220,24,M
985468,C8020229,26,M
985469,C6459278,24,M


In [28]:
df=df[~df['CustomerID'].isin(faulty)]

In [29]:
df

,CustomerID,CustomerDOB,CustGender,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),age
0,C5841053,10/1/94,F,17819.05,2/8/16,143207,25.0,22
1,C2142763,4/4/57,M,2270.69,2/8/16,141858,27999.0,59
2,C4417068,26/11/96,F,17874.44,2/8/16,142712,459.0,20
4,C9031234,24/3/88,F,6714.43,2/8/16,181156,1762.5,28
5,C1536588,8/10/72,F,53609.20,2/8/16,173940,676.0,44
...,...,...,...,...,...,...,...,...
1048560,C5028150,24/5/80,M,464.87,18/9/16,184632,3000.0,36
1048561,C1034220,19/11/92,M,1212.70,18/9/16,184734,1500.0,24
1048562,C8020229,8/4/90,M,7635.19,18/9/16,184824,799.0,26
1048563,C6459278,20/2/92,M,27311.42,18/9/16,183734,460.0,24


In [30]:
customers.drop_duplicates(inplace=True)

/tmp/ipykernel_19/2534375925.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers.drop_duplicates(inplace=True)


In [31]:
customers

,ID,Age,Gender
0,C5841053,22,F
1,C2142763,59,M
2,C4417068,20,F
4,C9031234,28,F
5,C1536588,44,F
...,...,...,...
985466,C5028150,36,M
985467,C1034220,24,M
985468,C8020229,26,M
985469,C6459278,24,M


In [32]:
customers.to_csv("Customers_data.csv")

# Cleaning and making transactions data

In [33]:
df.head()

,CustomerID,CustomerDOB,CustGender,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),age
0,C5841053,10/1/94,F,17819.05,2/8/16,143207,25.0,22
1,C2142763,4/4/57,M,2270.69,2/8/16,141858,27999.0,59
2,C4417068,26/11/96,F,17874.44,2/8/16,142712,459.0,20
4,C9031234,24/3/88,F,6714.43,2/8/16,181156,1762.5,28
5,C1536588,8/10/72,F,53609.20,2/8/16,173940,676.0,44


In [34]:
df=df.drop("CustGender",axis=1)
df=df.drop("CustomerDOB",axis=1)
df=df.drop("age",axis=1)

In [35]:
df=df.drop("TransactionDate",axis=1)

In [36]:
df

,CustomerID,CustAccountBalance,TransactionTime,TransactionAmount (INR)
0,C5841053,17819.05,143207,25.0
1,C2142763,2270.69,141858,27999.0
2,C4417068,17874.44,142712,459.0
4,C9031234,6714.43,181156,1762.5
5,C1536588,53609.20,173940,676.0
...,...,...,...,...
1048560,C5028150,464.87,184632,3000.0
1048561,C1034220,1212.70,184734,1500.0
1048562,C8020229,7635.19,184824,799.0
1048563,C6459278,27311.42,183734,460.0


In [37]:
df.to_csv("Transaction_details.csv")